# MCP Introduction with LangChain

This notebook demonstrates connecting to MCP servers and integrating them with LangChain agents using the `langchain-mcp-adapters` package.

## Setup

!pip install langchain langchain-openai langchain-mcp-adapters langgraph mcp python-dotenv -q

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please create a .env file with your API key.")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## LangChain Integration with MCP

### Using langchain-mcp-adapters

LangChain provides the `langchain-mcp-adapters` package, which offers a seamless way to connect LangChain applications to MCP servers. This package includes `MultiServerMCPClient`, which allows you to connect to multiple MCP servers and load their tools, resources, and prompts into LangChain agents.

### Key Benefits

- **Standardized Protocol**: Work with one protocol instead of juggling multiple APIs
- **Multiple Servers**: Connect to multiple MCP servers simultaneously
- **LangChain Native**: Tools, resources, and prompts are automatically LangChain-compatible
- **Stateless by Default**: Each tool call creates a fresh session, no context manager needed
- **Tool Interceptors**: Middleware pattern for customizing tool behavior

### The LangChain Documentation MCP Server

We'll use the LangChain documentation MCP server as our example:
- **MCP Server URL**: `https://docs.langchain.com/mcp`
- **Documentation Index**: `https://docs.langchain.com/llms.txt`
- **Transport**: HTTP (not stdio)
- **Provides**: Access to LangChain, LangGraph, and LangSmith documentation

This MCP server allows you to query the latest LangChain documentation programmatically, making it perfect for building agents that can answer questions about LangChain features and APIs.

### Connecting to the LangChain Documentation MCP Server

Now let's connect to the LangChain documentation MCP server using HTTP transport.

`MultiServerMCPClient` accepts a dictionary where keys are server names and values are configuration dicts specifying `transport` and connection details:

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import asyncio

# Configure the LangChain documentation MCP server
# MultiServerMCPClient takes a dict of server configs
# Each config specifies transport type and connection details
mcp_client = MultiServerMCPClient({
    "langchain-docs": {
        "transport": "http",
        "url": "https://docs.langchain.com/mcp"
    }
})

print("MCP client configured for LangChain documentation server")
print(f"Server: langchain-docs → https://docs.langchain.com/mcp")

### Loading MCP Tools into LangChain

The `get_tools()` method retrieves all tools from connected MCP servers and converts them to LangChain-compatible tools.

**Note:** `MultiServerMCPClient` is stateless by default — each call to `get_tools()` internally creates and tears down fresh sessions automatically. No `async with` context manager is needed:

In [ ]:
# Load tools from the MCP server
# The client is stateless: get_tools() creates ephemeral sessions under the hood
mcp_tools = await mcp_client.get_tools()

print(f"Loaded {len(mcp_tools)} tools from MCP server(s)")
print("\nAvailable tools:")
for tool in mcp_tools:
    print(f"  - {tool.name}: {tool.description[:80]}...")

### Using MCP Tools with LangChain Agents

Now let's create a LangChain agent that can use these MCP tools to query the LangChain documentation.

We use `create_react_agent` from `langgraph.prebuilt` with the `prompt` parameter to set the system message:

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

# Get tools from MCP server
tools = await mcp_client.get_tools()

# Create agent with the model and MCP tools
# Use the 'prompt' parameter (not 'state_modifier', which is deprecated)
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant that can answer questions about LangChain, LangGraph, and LangSmith by searching the official documentation. Always provide accurate, up-to-date information based on the documentation."
)

print(f"Agent created with {len(tools)} MCP tools")
print(f"Tools: {[tool.name for tool in tools]}")

### Querying LangChain Documentation

Let's test the agent by asking it questions about LangChain:

In [ ]:
# Query the agent about LangChain
question = "How do I create a LangChain agent with tools?"
print(f"Question: {question}\n")

result = await agent.ainvoke({
    "messages": [HumanMessage(content=question)]
})

print(f"Answer: {result['messages'][-1].content}")

### Accessing MCP Resources

MCP resources provide read-only access to data. The `get_resources()` method returns LangChain `Blob` objects.

You can optionally filter by server name or specific URIs:

In [ ]:
# Access MCP resources
# get_resources() is also stateless — no context manager needed
mcp_resources = await mcp_client.get_resources()


In [ ]:
mcp_tools = await mcp_client.get_tools()

In [ ]:
mcp_tools

### Middleware and Tool Interceptors

One of the powerful features of `langchain-mcp-adapters` is the ability to use tool interceptors (middleware) to customize tool behavior. This allows you to:

- **Log tool calls**: Track when and how tools are used
- **Modify requests/responses**: Transform tool inputs or outputs
- **Add validation**: Ensure tool inputs meet certain criteria
- **Implement retry logic, caching, or rate limiting**

Interceptors follow a handler callback pattern (similar to middleware in web frameworks). Any async callable matching the `(request, handler) -> result` signature works. Interceptors compose in an "onion" pattern where the first interceptor is the outermost layer.

Let's create a simple tool interceptor that logs tool usage:

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Tool interceptors are async callables with signature:
#   async def interceptor(request, handler) -> result
# - request: MCPToolCallRequest with .name and .arguments
# - handler: callable to invoke the next interceptor or the actual tool
# - return: MCPToolCallResult

async def logging_interceptor(request, handler):
    """Interceptor that logs tool calls and responses."""
    print(f"[LOG] Tool '{request.name}' called with input: {request.args}")
    result = await handler(request)
    print(f"[LOG] Tool '{request.name}' returned result: {str(result.content)[:100]}...")
    return result

# Create MCP client with tool interceptor
# Pass interceptors as a list via the tool_interceptors parameter
mcp_client_with_logging = MultiServerMCPClient(
    {
        "langchain-docs": {
            "transport": "http",
            "url": "https://docs.langchain.com/mcp"
        }
    },
    tool_interceptors=[logging_interceptor]
)

print("MCP client configured with logging interceptor")

### Complete Example: Agent with Logging

Let's create a complete example that uses the logging interceptor:

In [ ]:
# Get tools (interceptor will log tool calls)
tools = await mcp_client_with_logging.get_tools()

# Create agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant that answers questions about LangChain documentation."
)

# Ask a question
question = "What is LangChain and what are its main features?"
print(f"\nQuestion: {question}\n")
print("=" * 60)

result = await agent.ainvoke({
    "messages": [HumanMessage(content=question)]
})

print("=" * 60)
print(f"\nFinal Answer:\n{result['messages'][-1].content}")

### Key Takeaways

1. **langchain-mcp-adapters** provides a seamless way to integrate MCP servers with LangChain
2. **MultiServerMCPClient** accepts a dict of server configurations — no separate parameter classes needed
3. **Stateless by default** — each `get_tools()` or `get_resources()` call handles sessions internally, no `async with` needed
4. **HTTP Transport** is used for web-based MCP servers like the LangChain documentation server
5. **Tool Interceptors** are async callables with `(request, handler) -> result` signature, passed via `tool_interceptors=`
6. **`create_react_agent`** uses the `prompt` parameter (not `state_modifier`, which is deprecated)

### Documentation References

- **LangChain Documentation MCP**: `https://docs.langchain.com/mcp`
- **Documentation Index**: `https://docs.langchain.com/llms.txt`
- **langchain-mcp-adapters**: [LangChain MCP Documentation](https://docs.langchain.com/oss/python/langchain/mcp)
- **API Reference**: [langchain-mcp-adapters Reference](https://reference.langchain.com/python/langchain_mcp_adapters/)

### Next Steps

- Explore other MCP servers from the [official MCP servers repository](https://github.com/modelcontextprotocol/servers)
- Build custom tool interceptors for your specific use cases
- Combine multiple MCP servers for comprehensive agent capabilities